In [1]:
from datetime import datetime
import logging
import sys
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback

import warnings
warnings.filterwarnings('ignore')

optuna_dir = "..//optuna"

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')

train_df = pd.concat([train_df, train_extra_df], ignore_index=True)

train_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity', 'price'
]
test_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity',
]

cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert all to be the same type
train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

target = "price"
X = train_df.drop(columns=[target])
y = train_df[target]

# Only keeping important columns
model_features = [
    'weight_capacity', 'color', 'compartments', 'brand', 'material', 'is_waterproof'
]
X = X[model_features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .01, 0.05),
        'max_bin': trial.suggest_int('max_bin', 100000, 130000),
        'max_depth': 10,
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80),
        'min_split_gain': 0.5,
        'n_estimators': trial.suggest_int('n_estimators', 9000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 900, 1200),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.5, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.000001, 0.00001),
        'subsample': trial.suggest_float('subsample', 0.2, 0.25),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=700,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-02-05 17:35:41,606] A new study created in RDB with name: 2025-02-05_17-35-41


A new study created in RDB with name: 2025-02-05_17-35-41


[I 2025-02-05 17:47:23,458] Trial 0 finished with value: 38.77757827534952 and parameters: {'colsample_bytree': 0.30528771575740377, 'learning_rate': 0.023729726287148156, 'max_bin': 113784, 'min_child_samples': 41, 'n_estimators': 14347, 'num_leaves': 1194, 'reg_alpha': 1.1145243421358724, 'reg_lambda': 9.332164118064479e-06, 'subsample': 0.24379261778941813}. Best is trial 0 with value: 38.77757827534952.


Trial 0 finished with value: 38.77757827534952 and parameters: {'colsample_bytree': 0.30528771575740377, 'learning_rate': 0.023729726287148156, 'max_bin': 113784, 'min_child_samples': 41, 'n_estimators': 14347, 'num_leaves': 1194, 'reg_alpha': 1.1145243421358724, 'reg_lambda': 9.332164118064479e-06, 'subsample': 0.24379261778941813}. Best is trial 0 with value: 38.77757827534952.


[I 2025-02-05 18:01:25,592] Trial 1 finished with value: 38.78724723779155 and parameters: {'colsample_bytree': 0.33709912142446513, 'learning_rate': 0.016458940912505264, 'max_bin': 100185, 'min_child_samples': 65, 'n_estimators': 12938, 'num_leaves': 1089, 'reg_alpha': 0.9001090206907691, 'reg_lambda': 3.429333048648549e-06, 'subsample': 0.23508584066534363}. Best is trial 0 with value: 38.77757827534952.


...

Trial 47 pruned. Trial was pruned at iteration 0.


[I 2025-02-05 19:54:34,947] Trial 48 pruned. Trial was pruned at iteration 0.


Trial 48 pruned. Trial was pruned at iteration 0.


[I 2025-02-05 19:54:35,464] Trial 49 pruned. Trial was pruned at iteration 0.


Trial 49 pruned. Trial was pruned at iteration 0.


In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
import os


In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 500, 1500),
        'max_depth': trial.suggest_int('max_depth', 6, 40),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 10000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
        'subsample': trial.suggest_float('subsample', 0.05, .5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.15, .5),
        'max_bin': trial.suggest_int('max_bin', 500, 10000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.4, 0.6),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=200, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 400, 2000),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', .05, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 4000, 20000),
        'min_child_samples': trial.suggest_int('min_child_samples', 100, 300),
        'subsample': trial.suggest_float('subsample', 0.15, .25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'max_bin': trial.suggest_int('max_bin', 500, 50000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=300, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 400, 2000),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', .05, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 4000, 20000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.15, .25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'max_bin': trial.suggest_int('max_bin', 50000, 200000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=300, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
print("Best trial:")
best_trial = study.best_trial
print("  RMSE: {:.4f}".format(best_trial.value))
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print("    '{}': {}".format(key, value))

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .005, 0.15),
        'max_bin': trial.suggest_int('max_bin', 100000, 150000),
        'max_depth': trial.suggest_int('max_depth', 8, 11),
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 70),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'n_estimators': trial.suggest_int('n_estimators', 8000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 1500),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.1, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'subsample': trial.suggest_float('subsample', 0.2, 0.3),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=400, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .0005, 0.15),
        'max_bin': trial.suggest_int('max_bin', 100000, 150000),
        'max_depth': trial.suggest_int('max_depth', 8, 11),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 70),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'n_estimators': trial.suggest_int('n_estimators', 8000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 1500),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.1, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'subsample': trial.suggest_float('subsample', 0.2, 0.3),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=500, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

## Older stuff

In [ ]:
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'random_state': 42,
        # Narrowed ranges for faster convergence (you can tweak these)
        'num_leaves': trial.suggest_int('num_leaves', 3, 512),
        'max_depth': trial.suggest_int('max_depth', 1, 16),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_bin': trial.suggest_int('max_bin', 63, 1011),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    gbm = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=500,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    
    preds = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    
    return rmse

# 6. Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
study.trials_dataframe().sort_values(by='value').head(30)

In [ ]:
study.trials_dataframe().sort_values(by='value')

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


study_1 = optuna.create_study(direction='minimize')
study_1.optimize(objective, n_trials=100)

In [ ]:
study.trials_dataframe().sort_values(by='value')

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


study_1 = optuna.create_study(direction='minimize')
study_1.optimize(objective, n_trials=100)

In [ ]:
study_1.trials_dataframe().sort_values(by='value')

In [ ]:
print("Best trial:")
best_trial = study_1.best_trial
print("  RMSE: {:.4f}".format(best_trial.value))
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))